```arduino
flask-sqlite3-test/
├── instance/
│   └── test.db
├── templates/
│   ├── index.html
│   ├── group_modal.html
│   └── user_modal.html
├── static/
│   └── js/
│       ├── group_modal.js
│       ├── user_modal.js
│       └── index.js
├── db_init.py
└── app.py
```

In [10]:
from datetime import datetime, timezone

# 确保日期格式为 <class 'datetime.datetime'>
def date_for_sqlite(meta_date:str) -> datetime:
    if meta_date == '':
        new_date = datetime.now(timezone.utc)
    else:
        new_date = datetime.strptime(meta_date, '%Y-%m-%d')
    return new_date

date_for_sqlite('2024-05-18')

datetime.datetime(2024, 5, 18, 0, 0)

In [3]:
from app import app, db
from app import Group, User

from sqlalchemy.exc import OperationalError

with app.app_context():

    try:
        # 读取表格内容为 list，其中的元素都为各自对应模型的实例
        group_list = Group.query.all()
        user_list = User.query.all()
    except OperationalError as e:
        print(f"OperationalError: {e}")


#print(vars(group_list[0]))
#print(vars(user_list[0]))

In [4]:
for group in group_list:
    print(
        f"{group.id}|{group.name}|{group.created_on}"
    )

1|Group A|2024-05-31 19:24:25.547162
2|Group B|2024-05-22 19:24:25.547162
3|Group C|2024-04-10 19:24:25.547162


In [11]:
for user in user_list:
    print(
        f"{user.id}|{user.name}|{user.registered_on} ({type(user.registered_on)})|{user.group_id}"
    )

1|Peter|2024-06-01 19:24:25.555666 (<class 'datetime.datetime'>)|0
2|Stewie|2024-05-30 19:24:25.555666 (<class 'datetime.datetime'>)|0
3|Brian|2024-06-03 19:24:25.555666 (<class 'datetime.datetime'>)|2
4|Lois|2024-06-01 19:24:25.555666 (<class 'datetime.datetime'>)|0
5|Meg|2024-06-03 19:24:25.555666 (<class 'datetime.datetime'>)|0
6|Chris|2024-05-31 19:24:25.555666 (<class 'datetime.datetime'>)|0


In [6]:
import pandas as pd

user_data = [{'id': user.id, 
              'name': user.name, 
              'registered_on': user.registered_on,
              'group_id': user.group_id} 
              for user in user_list]
# group_id 列因为可能有空值None存在导致被识别为浮点数NaN，进而导致整列变成浮点数
user_df = pd.DataFrame(user_data)
user_df['group_id'] = user_df['group_id'].astype(pd.Int64Dtype())

In [7]:
user_df

,id,name,registered_on,group_id
0,1,Peter,2024-06-01 19:24:25.555666,0
1,2,Stewie,2024-05-30 19:24:25.555666,0
2,3,Brian,2024-06-03 19:24:25.555666,2
3,4,Lois,2024-06-01 19:24:25.555666,0
4,5,Meg,2024-06-03 19:24:25.555666,0
5,6,Chris,2024-05-31 19:24:25.555666,0


In [21]:
from datetime import datetime, timezone, timedelta

# type <datetime.date>
A = datetime.strptime('2024-06-06', '%Y-%m-%d').date() + timedelta(days=1)
B = datetime(2024, 6, 6, tzinfo=timezone.utc) + timedelta(days=1)
A == B

False